In [131]:
import yfinance as yf
from ssk import ssk
from score import get_scores_df
import pandas as pd
from stocksymbol import StockSymbol
from returns import get_yearly_normalized_price, get_monthly_adjusted_price
from pairs import get_distance

In [2]:
ssm = StockSymbol(ssk)

# Get tickers of companies listed in NASDAQ
symbol_list = ssm.get_symbol_list(market="US")
nasdaq_list = [x['symbol'] for x in symbol_list if x['exchange'] == 'NASDAQ']

# Randomly sample 100 of them
nasdaq_list = list(pd.Series(nasdaq_list).sample(700, random_state=1))

In [3]:
# Get the adjusted prices and scores

yearly_normalized_price = get_yearly_normalized_price(ticker_list=nasdaq_list, start_year = 2009)
yearly_normalized_price.to_csv("normalized price trend.csv", index=False)

fundamental_scores = get_scores_df(ticker_list=nasdaq_list)

for variable in fundamental_scores:
    fundamental_scores[variable].to_csv("normalized {} trend.csv".format(variable), index=False)

scores:  {'ebtda-margin': [      AGEN
year      
2009   NaN
2010   NaN
2011   NaN
2012   NaN
2013   NaN
2014   NaN
2015   NaN
2016   NaN
2017   NaN
2018   NaN
2019   NaN
2020   NaN
2021   NaN
2022   NaN,       CPLP
year      
2009   NaN
2010   NaN
2011   NaN
2012   NaN
2013   NaN
2014   NaN
2015   NaN
2016   NaN
2017   NaN
2018   NaN
2019   NaN
2020   NaN
2021   NaN
2022   NaN,       PAGP
year      
2010   NaN
2011   NaN
2012   NaN
2013   NaN
2014   NaN
2015   NaN
2016   NaN
2017   NaN
2018   NaN
2019   NaN
2020   NaN
2021   NaN
2022   NaN,       MDWD
year      
2011   NaN
2012   NaN
2013   NaN
2014   NaN
2015   NaN
2016   NaN
2017   NaN
2018   NaN
2019   NaN
2020   NaN
2021   NaN
2022   NaN,       TATT
year      
2009   NaN
2010   NaN
2011   NaN
2012   NaN
2013   NaN
2014   NaN
2015   NaN
2016   NaN
2017   NaN
2018   NaN
2019   NaN
2020   NaN
2021   NaN
2022   NaN,       PCYO
year      
2009   NaN
2010   NaN
2011   NaN
2012   NaN
2013   NaN
2014   NaN
2015   NaN
2016   NaN
2017   NaN


In [17]:
import pandas as pd
import copy

def get_pair_distance(df, start_year_month=None, end_year_month=None):
    '''
    Receives a dataframe with index being the date (could be years) and 
    columns are the stocks then calculates the spread using sum of squared distance
    '''

    if(start_year_month != None):
        df = df[df.index >= start_year_month]
    if(end_year_month != None):
        df = df[df.index < end_year_month]

    df = df.dropna(axis=1)

    # df_corr = {}

    # # calculate the sum of squared difference for each column
    # for stock_1 in df:
    #     df_corr[stock_1] = {}
    #     for stock_2 in df:
    #         if stock_1 != stock_2:
    #             ssd = ((df[stock_1] - df[stock_2])**2).sum()
    #             df_corr[stock_1][stock_2] = ssd

    df_corr = df.corr()

    pairs = {}
    for col in df_corr:
        for row in df_corr.index:

            key = '_'.join([val for val in sorted([col, row])])

            # If we already have a pair (a, b), we don't want another
            # pair (b, a). We also don't want a pair of an asset and itself
            if(row != col and key not in pairs):
                pairs[key] = df_corr[row][col]

    return pd.Series(pairs).sort_values(ascending=False)

def get_distance(df1, df2_list = [], start_year_month=None, end_year_month=None):
    # because we don't want to alter the original df1 sent in this function
    df = copy.deepcopy(df1)
    
    # because if column has all values as NaN, multiplying by others will result
    # in all of them having NaN
    df = df.dropna(axis=1, how='all')

    # We expect df1 and df2 to have the same column names with the content
    # being for different variables. For instance, one df will have information
    # about normalized ebtda-margin, while another will have information about
    # normalized cost of goods sold
    for df2 in df2_list:
        df2 = df2.dropna(axis=1, how='all')
        for column in df1:
            try:
                df[column] = df[column] * df2[column]
            except:
                continue

    return get_pair_distance(df, start_year_month, end_year_month)


In [22]:
import pandas as pd
import copy

def get_pair_distance(df, start_year_month=None, end_year_month=None):
    '''
    Receives a dataframe with index being the date (could be years) and 
    columns are the stocks then calculates the spread using sum of squared distance
    '''

    if(start_year_month != None):
        df = df[df.index >= start_year_month]
    if(end_year_month != None):
        df = df[df.index < end_year_month]

    df = df.dropna(axis=1)

    # df_corr = {}

    # # calculate the sum of squared difference for each column
    # for stock_1 in df:
    #     df_corr[stock_1] = {}
    #     for stock_2 in df:
    #         if stock_1 != stock_2:
    #             ssd = ((df[stock_1] - df[stock_2])**2).sum()
    #             df_corr[stock_1][stock_2] = ssd

    df_corr = df.corr()

    pairs = {}
    for col in df_corr:
        for row in df_corr.index:

            key = '_'.join([val for val in sorted([col, row])])

            # If we already have a pair (a, b), we don't want another
            # pair (b, a). We also don't want a pair of an asset and itself
            if(row != col and key not in pairs):
                pairs[key] = df_corr[row][col]

    return pd.Series(pairs).sort_values(ascending=False)

def get_distance(df1, df2_list = [], start_year_month=None, end_year_month=None):
    # because we don't want to alter the original df1 sent in this function
    df = copy.deepcopy(df1)
    
    # because if column has all values as NaN, multiplying by others will result
    # in all of them having NaN
    df = df.dropna(axis=1, how='all')

    # We expect df1 and df2 to have the same column names with the content
    # being for different variables. For instance, one df will have information
    # about normalized ebtda-margin, while another will have information about
    # normalized cost of goods sold
    for df2 in df2_list:
        df2 = df2.dropna(axis=1, how='all')
        for column in df1:
            try:
                df[column] = df[column] * df2[column]
            except:
                continue

    return get_pair_distance(df, start_year_month, end_year_month)


In [23]:
# Calculate the Sum of Squared distance on a one by one basis

# (1) Get the distance for the price
price_pairs = get_distance(yearly_normalized_price)
price_pairs.to_csv("distance price.csv")


# (2) Get the distance for the other variables
fundamental_variable_pairs = {}
for variable in fundamental_scores:
    fundamental_pairs = get_distance(fundamental_scores[variable])
    fundamental_variable_pairs[variable] = fundamental_pairs
    fundamental_pairs.to_csv("distance {}.csv".format(variable))


# (3) Combine the distances
price_pairs.name = 'price-pairs'
pairs = pd.concat([fundamental_variable_pairs[col] for col in fundamental_variable_pairs], axis=1)
pairs.columns = [col for col in fundamental_variable_pairs]
pairs = pairs.merge(price_pairs, how='left', left_index=True, right_index=True)

pairs.to_csv('distance combined one variable method.csv')

# (4) Here is how the results look like
pairs.head()

,ebtda-margin,gross-profit-margin,cogs-margin,eps-earnings-per-share-diluted,all-fundamental-variables,price-pairs
AACG_GURE,NaN,0.993198,0.993199,0.586029,-0.586037,NaN
PEBO_UBOH,NaN,0.992070,0.992069,0.643873,0.643874,0.901189
CVBF_FCCO,NaN,0.990282,0.990282,-0.557460,0.557460,NaN
FBIZ_UBOH,NaN,0.987049,0.987048,0.769316,0.769318,0.768724
AUB_HAFC,NaN,0.986588,0.986588,-0.661878,0.661878,0.795130


In [2]:
# Calculate the Sum of Squared distance by first combining
# information provided from each of the stocks. e.g multiplying
# the value of the normalized value of a variable such as ebtda-margin
# with another variable e.g cost-of-goods-sold margin and using 
# the new value to calculate the sum of squared distance

# (1) Get the Sum of squared difference for combined everything
# this is calculated by normalized value price * ebtda * cost-of-goods-sold ...
# then calculating the sum of squared difference between each stock
# Please note that columns where all values are NaN are dropped before
# any calculation is done
price_pairs_2 = get_distance(yearly_normalized_price, [fundamental_scores[variable] for variable in fundamental_scores])
price_pairs_2.to_csv("distance price & all variables.csv")


# (2) Get the distance for the other variables
fundamental_variable_pairs = {}
for variable in fundamental_scores:
    fundamental_pairs = get_distance(yearly_normalized_price, [fundamental_scores[variable]])
    fundamental_variable_pairs[variable] = fundamental_pairs
    fundamental_pairs.to_csv("distance {} and price.csv".format(variable))


# (3) Combine the distances
price_pairs_2.name = 'price-pairs'
pairs = pd.concat([fundamental_variable_pairs[col] for col in fundamental_variable_pairs], axis=1)
pairs.columns = ["distance {} and price".format(col) for col in fundamental_variable_pairs]
# pairs = pairs.merge(price_pairs_2, how='left', left_index=True, right_index=True)

# pairs.to_csv('distance combined multiple variable method.csv')

# (4) Here is how the results look like
pairs.head()

NameError: name 'get_distance' is not defined

In [3]:
a = 'distance all-fundamental-variables and price.csv'
b = 'distance cogs-margin and price.csv'
c = 'distance ebtda-margin and price.csv'
d = 'distance eps-earnings-per-share-diluted and price.csv'
e = 'distance gross-profit-margin and price.csv'

In [80]:
import pandas as pd


# x = pd.concat([pd.read_csv(x) for x in [a, b, c, d]], axis=1)

dfs = []
for x in [a, b, c, d]:
    df = pd.read_csv(x)
    df.rename(columns= {'0': x.split(" ")[1], 'Unnamed: 0': 'stock pair'}, inplace=True)
    df = df.set_index('stock pair')
    dfs.append(df)

x = pd.concat(dfs, axis=1)
x.to_csv('distance combined multiple variable method.csv')
x['all-fundamental-variables'] = abs(x['all-fundamental-variables'])

y = x['all-fundamental-variables'].dropna()
y = y.sort_values(ascending=False)

top_ten_method_2 = y.head(10)
bottom_ten_method_2 = y.tail(10)

top_ten_method_2.to_csv('top ten method 2.csv')
bottom_ten_method_2.to_csv('bottom ten method 2.csv')


In [61]:
method1_df = pd.read_csv('distance combined one variable method.csv')
method1_df = method1_df[(method1_df['price-pairs'] > 0) & (method1_df['all-fundamental-variables'] > 0)]
method1_df['pairs'] = method1_df['price-pairs'] * method1_df['all-fundamental-variables']

method1_df.rename(columns={'Unnamed: 0': 'stock pair'}, inplace=True)
method1_df = method1_df.set_index('stock pair')

method1_pairs = abs(method1_df['pairs'])
method1_pairs.dropna(inplace=True)
method1_pairs.sort_values(ascending=False, inplace=True)

top_ten_method_1 = method1_pairs.head(10)
bottom_ten_method_1 = method1_pairs.tail(10)

top_ten_method_1.to_csv('top ten method 1.csv')
bottom_ten_method_1.to_csv('bottom ten method 1.csv')

In [82]:
df2 = pd.read_csv("distance combined one variable method.csv")
df2.head()

,Unnamed: 0,ebtda-margin,gross-profit-margin,cogs-margin,eps-earnings-per-share-diluted,all-fundamental-variables,price-pairs
0,AACG_GURE,NaN,0.993198,0.993199,0.586029,-0.586037,NaN
1,PEBO_UBOH,NaN,0.992070,0.992069,0.643873,0.643874,0.901189
2,CVBF_FCCO,NaN,0.990282,0.990282,-0.557460,0.557460,NaN
3,FBIZ_UBOH,NaN,0.987049,0.987048,0.769316,0.769318,0.768724
4,AUB_HAFC,NaN,0.986588,0.986588,-0.661878,0.661878,0.795130


In [109]:
price = pd.read_csv("distance price.csv")
price.rename(columns={'Unnamed: 0': 'pair', '0': 'distance'}, inplace=True)
price = price.set_index('pair')
price['distance'] = abs(price['distance'])
price['distance'] = price['distance'].sort_values()

price['distance'] = price['distance'].sort_values(ascending=False)
top_ten_traditional = price.head(10)
top_ten_traditional.to_csv('top ten price distance traditional.csv')

In [124]:
top_ten_method_1
top_ten_traditional = top_ten_traditional['distance']
top_ten_method_2
top_ten_traditional

pair
GOOGL_INTU    0.990578
AUB_ONB       0.990565
CIVB_FCCO     0.984932
CIVB_PFC      0.984857
EFSC_FCCO     0.984058
ESLT_LMAT     0.983733
CIVB_EFSC     0.982317
AROW_TRMK     0.982065
CSWC_GOOGL    0.980652
CCOI_SBUX     0.980311
Name: distance, dtype: float64

In [312]:
## We get a list of stocks from the pairs calculated above

def get_stocks(pair_list):
    result = []
    for pair in pair_list:
        result.append(pair.split('_')[0])
        result.append(pair.split('_')[1])

    return list(set(result))

pairs_method_1 = get_stocks(top_ten_method_1.keys())
pairs_method_2 = get_stocks(top_ten_method_2.keys())
pairs_traditional = get_stocks(top_ten_traditional.keys())
all_stocks = list(set(pairs_method_1 + pairs_method_2 + pairs_method_3))
all_stocks[:5]

['CIVB', 'CCOI', 'FCCO', 'IMKTA', 'AROW']

In [136]:
pairs_data = get_monthly_adjusted_price(all_stocks, start_year=2009)
pairs_data.to_csv("pairs data.csv", index=False)

In [212]:
# Get the relative prices

def get_relative_price(pairs, pairs_data):
    stock_pairs_arr = []
    for pair in pairs.keys():
        stock1 = pair.split("_")[0]
        stock2 = pair.split("_")[1]

        relative_price = pairs_data[stock1]/pairs_data[stock2]
        relative_price.name = pair
        stock_pairs_arr.append(relative_price)

    return pd.concat(stock_pairs_arr, axis=1)

relative_prices_method_1 = get_relative_price(top_ten_method_1, pairs_data)
relative_prices_method_2 = get_relative_price(top_ten_method_2, pairs_data)
relative_prices_traditional = get_relative_price(top_ten_traditional, pairs_data)

relative_prices_method_1.head()

AUB_PFC  EBTC_FNLC  CBSH_PFC  GOOGL_NFLX  EXLS_IMKTA  AUB_CBSH  \
year month                                                                    
2009 1      1.000000   1.000000  1.000000    1.000000    1.000000  1.000000   
     2      1.064330   1.260456  1.317906    0.995650    1.068565  0.807592   
     3      1.008749   0.899864  1.183076    0.865736    1.035618  0.852649   
     4      0.719140   1.020789  0.623264    0.932952    1.058505  1.153828   
     5      0.445195   1.159314  0.429971    1.129926    1.176064  1.035407   

            EBTC_HBNC  BRKL_PFC  FELE_NSIT  CBSH_FNLC  
year month                                             
2009 1       1.000000  1.000000   1.000000   1.000000  
     2       0.975746  1.240071   1.674649   1.358774  
     3       0.992764  1.143146   1.447827   1.051652  
     4       0.902405  0.695490   0.829136   0.861681  
     5       0.793943  0.502175   0.633683   0.842225

In [196]:
# Get data summary

def filter_data(df, min_year=2009, min_month=1, max_year=2023, max_month=12):
    period_df = df[(df.index >= (min_year, min_month)) & (df.index < (max_year, max_month))]

    return period_df

def get_summary(df, min_year=2009, min_month=1, max_year=2016, max_month=12):
    data = filter_data(df, min_year, min_month, max_year, max_month)
    return data.describe()

In [197]:
summary_method_1 = get_summary(relative_prices_method_1)
summary_method_1

,AUB_PFC,EBTC_FNLC,CBSH_PFC,GOOGL_NFLX,EXLS_IMKTA,AUB_CBSH,EBTC_HBNC,BRKL_PFC,FELE_NSIT,CBSH_FNLC
count,95.000000,95.000000,95.000000,95.000000,95.000000,95.000000,95.000000,95.000000,95.000000,95.000000
mean,0.423316,1.552284,0.507285,0.511655,1.755958,0.861938,0.613368,0.421809,0.620538,1.170059
std,0.141772,0.249681,0.192622,0.338735,0.486212,0.138005,0.156768,0.207335,0.183860,0.129057
min,0.272472,0.899864,0.291560,0.171606,0.950682,0.585047,0.441800,0.219627,0.410453,0.748376
25%,0.342319,1.392368,0.358948,0.252650,1.413532,0.743339,0.505924,0.258588,0.501459,1.079076
50%,0.401689,1.580602,0.447511,0.341169,1.728181,0.897475,0.534997,0.350221,0.565591,1.201857
75%,0.439547,1.723493,0.601757,0.818321,2.136278,0.959208,0.700425,0.509509,0.696389,1.258840
max,1.064330,2.056830,1.317906,1.479558,2.779984,1.153828,1.112004,1.240071,1.674649,1.389974


In [265]:
# Get the time for opening and closing the long-short position in the
# stock pair
def get_entry_and_exit_points(relative_prices, mean, std):
    result = []
    next_position = 'open'
    trade = {}
    for rp, date in zip(relative_prices, relative_prices.index):
        big = rp > mean + (2 * std)
        small = rp < mean - (2 * std)
        if (big or small) and (next_position == 'open'):
            trade['start'] = date
            next_position = 'close'
        elif (abs(rp) < abs(mean + (2 * std))) and (next_position == 'close'):
            trade['end'] = date
            result.append(trade)

            next_position = 'open'
            trade = {}

    return result

# Get the relative prices, mean, and standard deviation of the stocks
def get_pair_info(pair, relative_prices):
    pair_summary = get_summary(relative_prices)
    relative_price = filter_data(relative_prices, min_year=2017)[pair]
    std = pair_summary.loc['std', pair]
    mean = pair_summary.loc['mean', pair]

    return {"relative price": relative_price, "mean": mean, "std": std}

Get the trades that we should make for each of the pairs

In [266]:
def get_trades(relative_prices):
    result = {}
    for pair in relative_prices.columns:
        
        pair_info = get_pair_info(pair, relative_prices)
        rp = pair_info['relative price']
        mean = pair_info['mean']
        std = pair_info['std']

        entry_and_exit = get_entry_and_exit_points(rp, mean, std)

        result[pair] = entry_and_exit

    return result

def get_trade_count(trades):
    no_of_trades = {}
    for pair in trades:
        no_of_trades[pair] = len(trades[pair])

    return pd.Series(no_of_trades)

In [267]:
method_1_trades = get_trades(relative_prices_method_1)
method_1_trades

{'AUB_PFC': [],
 'EBTC_FNLC': [],
 'CBSH_PFC': [],
 'GOOGL_NFLX': [],
 'EXLS_IMKTA': [{'start': (2017, 8), 'end': (2017, 12)},
  {'start': (2019, 3), 'end': (2019, 5)},
  {'start': (2019, 7), 'end': (2019, 8)},
  {'start': (2020, 11), 'end': (2020, 12)}],
 'AUB_CBSH': [{'start': (2017, 11), 'end': (2017, 12)}],
 'EBTC_HBNC': [],
 'BRKL_PFC': [],
 'FELE_NSIT': [],
 'CBSH_FNLC': [{'start': (2017, 10), 'end': (2017, 11)},
  {'start': (2020, 5), 'end': (2020, 6)}]}

In [268]:
get_trade_count(method_1_trades)

AUB_PFC       0
EBTC_FNLC     0
CBSH_PFC      0
GOOGL_NFLX    0
EXLS_IMKTA    4
AUB_CBSH      1
EBTC_HBNC     0
BRKL_PFC      0
FELE_NSIT     0
CBSH_FNLC     2
dtype: int64

In [269]:
method_2_trades = get_trades(relative_prices_method_2)
get_trade_count(method_2_trades)

CHI_CSWC     2
CGO_CHI      9
CGO_HSKA     0
CPSS_ECPG    1
CHI_HSKA     0
CSWC_HSKA    0
CGO_CSWC     4
ARWR_CGO     1
CSWC_DHIL    0
ARWR_CHI     4
dtype: int64

In [270]:
traditional_trades = get_trades(relative_prices_traditional)
traditional_trades

{'GOOGL_INTU': [{'start': (2019, 2), 'end': (2019, 3)},
  {'start': (2019, 5), 'end': (2019, 6)},
  {'start': (2019, 7), 'end': (2019, 8)},
  {'start': (2019, 9), 'end': (2019, 10)},
  {'start': (2020, 8), 'end': (2020, 9)},
  {'start': (2020, 12), 'end': (2021, 1)},
  {'start': (2021, 10), 'end': (2021, 11)},
  {'start': (2021, 12), 'end': (2022, 1)},
  {'start': (2022, 10), 'end': (2022, 11)},
  {'start': (2022, 12), 'end': (2023, 1)},
  {'start': (2023, 2), 'end': (2023, 3)},
  {'start': (2023, 4), 'end': (2023, 5)}],
 'AUB_ONB': [{'start': (2017, 5), 'end': (2017, 6)},
  {'start': (2018, 1), 'end': (2018, 6)},
  {'start': (2019, 4), 'end': (2019, 5)},
  {'start': (2019, 6), 'end': (2019, 10)},
  {'start': (2021, 5), 'end': (2021, 6)},
  {'start': (2021, 7), 'end': (2021, 10)},
  {'start': (2022, 1), 'end': (2022, 7)},
  {'start': (2023, 1), 'end': (2023, 5)}],
 'CIVB_FCCO': [{'start': (2017, 9), 'end': (2017, 10)},
  {'start': (2019, 3), 'end': (2019, 11)},
  {'start': (2019, 12), 

In [271]:
get_trade_count(traditional_trades)

GOOGL_INTU    12
AUB_ONB        8
CIVB_FCCO      6
CIVB_PFC       0
EFSC_FCCO      3
ESLT_LMAT      0
CIVB_EFSC      0
AROW_TRMK      5
CSWC_GOOGL    13
CCOI_SBUX      3
dtype: int64

The Average Mean Reversion Period

In [335]:
def get_month_difference(start, end):
    '''
    Receives two tuples in the form (2009, 1), (2010, 3) and returns the
    difference between the two in number of months. The first vlaue is
    expected to be the smaller one
    '''
    year_difference = end[0] - start[0]

    return (year_difference * 12) + end[1] - start[1]

def get_convertence_time(pair_trades):
    
    trades = []
    for pair in pair_trades:
        trades += pair_trades[pair]

    trade_period = []
    for trade in trades:
        end = trade['end']
        start = trade['start']

        trade_period.append(get_month_difference(start, end))

    return trade_period

In [343]:
trade_period_traditional = get_convertence_time(traditional_trades)
pd.Series(trade_period_traditional).to_csv('trade period traditional.csv')
average_convergence_time_traditional = sum(trade_period_traditional)/len(trade_period_traditional)
average_convergence_time_traditional

2.14

In [344]:
trade_period_method_1 = get_convertence_time(method_1_trades)
pd.Series(trade_period_method_1).to_csv('trade period method 1.csv')
average_convergence_time_traditional = sum(trade_period_method_1)/len(trade_period_method_1)
average_convergence_time_traditional

1.5714285714285714

In [345]:
trade_period_method_2 = get_convertence_time(method_2_trades)
pd.Series(trade_period_method_2).to_csv('trade period method 2.csv')
average_convergence_time_traditional = sum(trade_period_method_2)/len(trade_period_method_2)
average_convergence_time_traditional

5.190476190476191

In [313]:
def get_profit(pair, price_df, trade_info):
    stock1 = pair.split('_')[0]
    stock2 = pair.split('_')[1]

    trades = trade_info[pair]

    result = []
    for trade in trades:
        stock1_start_price = price_df.loc[trade['start'], stock1]
        stock1_end_price = price_df.loc[trade['end'], stock1]
        stock2_start_price = price_df.loc[trade['start'], stock2]
        stock2_end_price = price_df.loc[trade['end'], stock2]

        stock1_difference = abs(stock1_end_price - stock1_start_price)/stock1_start_price
        stock2_difference = abs(stock2_end_price - stock2_start_price)/stock2_start_price

        return_val = abs(stock1_difference - stock2_difference)
        month_count = get_month_difference(trade['start'], trade['end'])

        result.append(return_val / month_count)

    return result

In [326]:
traditional_profits = []

for pair in relative_prices_traditional:
    traditional_profits += get_profit(pair, pairs_data, traditional_trades)

sum(traditional_profits)/len(traditional_profits)

0.03472318219111427

In [323]:
method_1_profits = []

for pair in relative_prices_method_1:
    method_1_profits += get_profit(pair, pairs_data, method_1_trades)

sum(method_1_profits)/len(method_1_profits)

0.07839440576439947

In [328]:
method_2_profits = []

for pair in relative_prices_method_2:
    method_2_profits += get_profit(pair, pairs_data, method_2_trades)

sum(method_2_profits)/len(method_2_profits)

0.03572954047576432

In [330]:
t = pd.Series(traditional_profits)
t.name = 'traditional'
t.to_csv('traditional profits.csv')
m1 = pd.Series(method_1_profits)
m1.name = 'method 1'
m1.to_csv('method 1 profits.csv')
m2 = pd.Series(method_2_profits)
m2.name = 'method 2'
m2.to_csv('method 2 profits.csv')